In [3]:
import chess.pgn
import io
from typing import List
from chess.pgn import Game

In [4]:
def load_first_chunk_of_pgn(file_path, chunk_size) -> List[Game]:
    """
    Loads the first chunk of games from a PGN file.
    
    :param file_path: Path to the PGN file.
    :param chunk_size: Number of games to load in the chunk.
    :return: List of games in the first chunk.
    """
    games: List[Game] = []
    with open(file_path) as pgn_file:
        for _ in range(chunk_size):
            try:
                game = chess.pgn.read_game(pgn_file)
                if game is None:  # End of file
                    break
                games.append(game)
            except Exception as e:
                print(f"Error reading game: {e}")
                continue
    return games

In [5]:
# Define your file path and chunk size
pgn_file_path = "dataset/lichess_db_standard_rated_2016-05.pgn"
chunk_size = 100  # for example, load the first 100 games

# Load the first chunk
first_chunk = load_first_chunk_of_pgn(pgn_file_path, chunk_size)
print(f"Loaded {len(first_chunk)} games in the first chunk")


Loaded 100 games in the first chunk


In [6]:
game = first_chunk[0]

In [12]:
vars(game.headers)

{'_tag_roster': {'Event': 'Rated Bullet tournament https://lichess.org/tournament/IaRkDsvp',
  'Site': 'https://lichess.org/r0cYFhsy',
  'Date': '????.??.??',
  'Round': '?',
  'White': 'GreatGig',
  'Black': 'hackattack',
  'Result': '0-1'},
 '_others': {'UTCDate': '2016.04.30',
  'UTCTime': '22:00:03',
  'WhiteElo': '1777',
  'BlackElo': '1809',
  'WhiteRatingDiff': '-11',
  'BlackRatingDiff': '+11',
  'ECO': 'B01',
  'Opening': 'Scandinavian Defense: Mieses-Kotroc Variation',
  'TimeControl': '60+0',
  'Termination': 'Time forfeit'}}

In [8]:
moves = [move.uci() for move in game.mainline_moves()]
print("Moves:", " ".join(moves))


Moves: e2e4 d7d5 e4d5 d8d5 b1c3 d5d8 d2d4 g8f6 g1f3 c8g4 h2h3 g4f3 g2f3 c7c6 f1g2 b8d7 c1e3 e7e6 d1d2 f6d5 c3d5 c6d5 e1c1 f8e7 c2c3 d8c7 c1b1 e8c8 f3f4 c8b8 h1g1 b8a8 g2h1 g7g6 h3h4 e7h4 f2f3 h4e7 d2c2 d7f6 h1g2 f6h5 g2h3 h5f4 e3f4 c7f4 d1f1 f4d6 g1g4 d8f8 f1g1 f7f5 g4g2 e7f6 g2g3 f8g8 h3f1 g8g7 f1d3 h8g8 c2h2 d6b8 h2g2 b8c8 f3f4 c8c6 g2f2 f6h4 g3g6 h4f2 g6g7 g8g7 g1g7 a7a6 g7g8 a8a7 g8h8 c6d7 h8h7 d7h7


In [9]:
game.headers["Result"]

'0-1'

In building the supervised learning part of the model, we'll focus on training a network to predict the next move in a given chess position.

We will therefore process the PGN dataset to extract board positions and corresponding moves. This will involve converting chessboard positions into a suitable numerical format for the CNN.

We will design a CNN that takes the board position as input. The output layer should predict the probability of each possible move. Since there are a limited number of legal moves in any given position, this becomes a multi-class classification problem.

### The Input Layer

The input layer should reflect the board layout (8x8 squares with channels representing different pieces).

### The Output Layer

The output should represent all possible moves. In chess, a common approach is to have a fixed-size array where each entry corresponds to a potential move.

Our first mission is to construct the input and output tensor:
X --> Y

Where X, the input is the board layout at every single game state and Y is the corresponding next move (output).

This is an example of the data for a given game:

```Moves: e2e4 d7d5 e4d5 d8d5 b1c3 d5d8 d2d4 g8f6 g1f3 c8g4 h2h3 g4f3 g2f3 c7c6 f1g2 b8d7 c1e3 e7e6 d1d2 f6d5 c3d5 c6d5 e1c1 f8e7 c2c3 d8c7 c1b1 e8c8 f3f4 c8b8 h1g1 b8a8 g2h1 g7g6 h3h4 e7h4 f2f3 h4e7 d2c2 d7f6 h1g2 f6h5 g2h3 h5f4 e3f4 c7f4 d1f1 f4d6 g1g4 d8f8 f1g1 f7f5 g4g2 e7f6 g2g3 f8g8 h3f1 g8g7 f1d3 h8g8 c2h2 d6b8 h2g2 b8c8 f3f4 c8c6 g2f2 f6h4 g3g6 h4f2 g6g7 g8g7 g1g7 a7a6 g7g8 a8a7 g8h8 c6d7 h8h7 d7h7```

To construct the input we will have to recreate the board layout for every single game state.

### X Tensor (Board State Representation)
The X tensor represents the state of the chess board.

X will be a 3-dimensional tensor, 2-dimensions representing the layout (8x8) and a third dimension represnting the type of chess piece discriminated by color. The tensor can be represented as follows:

$X \in \mathbb{R}^{8 \times 8 \times 12}$

Each element $X_{i,j,k}$ of this tensor can be defined as:

$$\ X_{i,j,k} = 
   \begin{cases} 
   1 & \text{if piece type } k \text{ is present at position } (i, j) \\
   0 & \text{otherwise}
   \end{cases}
\$$

So in essence we're dealing with a 3-dimensional tensor with binary states.

### Y Tensor (Next Move Representation)

There are 64 squares on a chessboard, so there are 64 possible starting points and 64 possible ending points for each move, leading to 64×64=4096 possible moves (including illegal ones, which the model should learn to never predict).

The Y tensor represents the next move for each given game state. We encode each move as a one-hot vector of length 64x64 (representing all possible source and destination squares), the tensor can be represented as:

$Y \in \{0, 1\}^{4096}$

Each element $Y_{l}$ of this tensor, where $l$ corresponds to a combination of source and destination squares, can be defined as:

$$Y_{l} = 
   \begin{cases} 
   1 & \text{if the move corresponds to the index } l \\
   0 & \text{otherwise}
   \end{cases}
\$$

In this representation, the index $l$ is calculated based on the source square and the destination square of the move. For instance, if you flatten the 8x8 board into a 64-element array, then a move from square $a$ to square $b$ would correspond to an index $l = 64 \times a + b$.

### Summary
- **X Tensor**: Represents the board state with a 3D tensor where the dimensions are board height, board width, and number of piece types.
- **Y Tensor**: Represents the move as a one-hot encoded vector in a flattened 2D space of source and destination squares.

In [13]:
import torch
import chess
from torch.utils.data import Dataset

class ChessDataset(Dataset):
    def __init__(self, games):
        self.positions = []
        self.moves = []
        self.process_games(games)

    def process_games(self, games):
        for game in games:
            board = chess.Board()
            for move in game.split():
                # Add current board state to positions
                self.positions.append(self.board_to_tensor(board))
                # Make move and add to moves
                chess_move = board.parse_san(move)
                self.moves.append(self.move_to_tensor(chess_move))
                board.push(chess_move)

    def board_to_tensor(self, board):
        # Convert board to 8x8xN tensor
        pass

    def move_to_tensor(self, move):
        # Convert move to tensor (e.g., one-hot encoding)
        pass

    def __len__(self):
        return len(self.positions)

    def __getitem__(self, idx):
        return self.positions[idx], self.moves[idx]

# Example usage - TODO
games = ["e2e4 e7e5 g1f3...", "d2d4 d7d5 c2c4..."]  # list of game move sequences
dataset = ChessDataset(games)


/Users/nuno/Library/Caches/pypoetry/virtualenvs/gammax-NkQe2Tz_-py3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


InvalidMoveError: invalid san: 'g1f3...'